# Experiment 1 - Scale everything with Min Max

In [1]:
from project_1.config import PROJ_ROOT, PROCESSED_DATA_DIR
from project_1.loading import *
from project_1.features import *
from project_1.dataset import *

import torch
import torch.nn as nn
torch.manual_seed(42)


2025-03-26 20:19:58.791 | INFO     | project_1.config:<module>:11 - PROJ_ROOT path is: /Users/francescobondi/Desktop/stuff/ETH/FS25/ML for Healthcare/project-1-ml4hc


In [2]:
set_a, set_b, set_c = load_before_scaling()

Shapes of the datasets:
Set A: (183416, 43) Set B: (183495, 43) Set C: (183711, 43)


In [3]:
# Scale using basic scaling
set_a_scaled, set_b_scaled, set_c_scaled = scale_features_basic(set_a, set_b, set_c)

# Remove the ICUType feature
set_a_scaled = set_a_scaled.drop(columns=['ICUType'])
set_b_scaled = set_b_scaled.drop(columns=['ICUType'])
set_c_scaled = set_c_scaled.drop(columns=['ICUType'])

In [4]:
# Load outcomes
death_a, death_b, death_c = load_outcomes()

Shapes of labels:
Set A: (4000, 2) Set B: (4000, 2) Set C: (4000, 2)


In [5]:
train_dataset = create_dataset_from_timeseries(set_a_scaled, death_a["In-hospital_death"])
validation_dataset = create_dataset_from_timeseries(set_b_scaled, death_b["In-hospital_death"])
test_dataset = create_dataset_from_timeseries(set_c_scaled, death_c["In-hospital_death"])

train_dataset.tensors[0].shape # (batch_size, seq_len, input_size)

torch.Size([4000, 49, 40])

In [6]:
# Convert to DataLoader
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Try directly with LSTM

In [ ]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=1, dropout=0.3):
        super(LSTM_Model, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,       # 41 features per time step
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)           # out: (batch_size, seq_len, hidden_size)
        out = out[:, -1, :]             # Take last time step: (batch_size, hidden_size)
        out = self.fc(out)              # (batch_size, num_classes)
        return out.squeeze()            # (batch_size,) for BCEWithLogitsLoss

# Train loop

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = train_dataset.tensors[0].shape[-1]
model = LSTM_Model(input_size=input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Call the trainig loop (default 10 epochs)
model = train_model_with_validation(model, train_loader, validation_loader, criterion, optimizer, device)

/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/20
  Train Loss: 0.4415 | AUCROC: 0.5029 | AUPRC: 0.1379
  Val   Loss: 0.4106 | AUCROC: 0.5484 | AUPRC: 0.1500



Epoch 2/20
  Train Loss: 0.3998 | AUCROC: 0.5559 | AUPRC: 0.1724
  Val   Loss: 0.3883 | AUCROC: 0.7539 | AUPRC: 0.3780



Epoch 3/20
  Train Loss: 0.3736 | AUCROC: 0.6914 | AUPRC: 0.2913
  Val   Loss: 0.3600 | AUCROC: 0.8085 | AUPRC: 0.4491



Epoch 4/20
  Train Loss: 0.3386 | AUCROC: 0.7803 | AUPRC: 0.3855
  Val   Loss: 0.3322 | AUCROC: 0.8193 | AUPRC: 0.4644



...

Epoch 16/20
  Train Loss: 0.3085 | AUCROC: 0.8299 | AUPRC: 0.4776
  Val   Loss: 0.3203 | AUCROC: 0.8380 | AUPRC: 0.5004



Epoch 17/20
  Train Loss: 0.3089 | AUCROC: 0.8286 | AUPRC: 0.4773
  Val   Loss: 0.3077 | AUCROC: 0.8379 | AUPRC: 0.5033



Epoch 18/20
  Train Loss: 0.3097 | AUCROC: 0.8272 | AUPRC: 0.4738
  Val   Loss: 0.3216 | AUCROC: 0.8394 | AUPRC: 0.5064



Epoch 19/20
  Train Loss: 0.3086 | AUCROC: 0.8270 | AUPRC: 0.4888
  Val   Loss: 0.3106 | AUCROC: 0.8365 | AUPRC: 0.5081



Epoch 20/20
  Train Loss: 0.3085 | AUCROC: 0.8311 | AUPRC: 0.4741
  Val   Loss: 0.3116 | AUCROC: 0.8359 | AUPRC: 0.5061



In [9]:
avg_loss, aucroc, auprc = evaluate_model(model, test_loader, criterion, device)
print(f"Test Loss: {avg_loss:.4f}, AUC-ROC: {aucroc:.4f}, AUC-PRC: {auprc:.4f}")

Evaluation - Loss: 0.3231 - AUCROC: 0.8311 - AUPRC: 0.4971
Test Loss: 0.3231, AUC-ROC: 0.8311, AUC-PRC: 0.4971


# LSTM w/ Max Pooling

In [10]:
class LSTM_Model_Max_Pooling(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=1, dropout=0.3):
        super(LSTM_Model_Max_Pooling, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,       # 40 features per time step
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)           # out: (batch_size, seq_len, hidden_size)
        out, _ = out.max(dim=1)           # Pooling: (batch_size, hidden_size)   
        out = self.fc(out)              # (batch_size, num_classes)
        return out.squeeze()            # (batch_size,) for BCEWithLogitsLoss

In [11]:
# Use the previous data loaders and train the new model
model_max_pooling = LSTM_Model_Max_Pooling(input_size=input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_max_pooling.parameters(), lr=0.001)

model_max_pooling = train_model_with_validation(model_max_pooling, train_loader, validation_loader, criterion, optimizer, device)

Epoch 1/20
  Train Loss: 0.4506 | AUCROC: 0.5164 | AUPRC: 0.1430
  Val   Loss: 0.4152 | AUCROC: 0.5970 | AUPRC: 0.1913



Epoch 2/20
  Train Loss: 0.4027 | AUCROC: 0.4995 | AUPRC: 0.1353
  Val   Loss: 0.4107 | AUCROC: 0.6563 | AUPRC: 0.2379



Epoch 3/20
  Train Loss: 0.4028 | AUCROC: 0.5000 | AUPRC: 0.1395
  Val   Loss: 0.4094 | AUCROC: 0.7092 | AUPRC: 0.2855



Epoch 4/20
  Train Loss: 0.3992 | AUCROC: 0.5892 | AUPRC: 0.2063
  Val   Loss: 0.4074 | AUCROC: 0.7631 | AUPRC: 0.3297



Epoch 5/20
  Train Loss: 0.3986 | AUCROC: 0.6155 | AUPRC: 0.1994
  Val   Loss: 0.3969 | AUCROC: 0.8031 | AUPRC: 0.4251



...

Epoch 16/20
  Train Loss: 0.3243 | AUCROC: 0.8058 | AUPRC: 0.4243
  Val   Loss: 0.3193 | AUCROC: 0.8293 | AUPRC: 0.4656



Epoch 17/20
  Train Loss: 0.3220 | AUCROC: 0.8091 | AUPRC: 0.4403
  Val   Loss: 0.3292 | AUCROC: 0.8297 | AUPRC: 0.4587



Epoch 18/20
  Train Loss: 0.3214 | AUCROC: 0.8115 | AUPRC: 0.4329
  Val   Loss: 0.3179 | AUCROC: 0.8310 | AUPRC: 0.4673



Epoch 19/20
  Train Loss: 0.3185 | AUCROC: 0.8153 | AUPRC: 0.4425
  Val   Loss: 0.3227 | AUCROC: 0.8292 | AUPRC: 0.4666



Epoch 20/20
  Train Loss: 0.3173 | AUCROC: 0.8169 | AUPRC: 0.4482
  Val   Loss: 0.3170 | AUCROC: 0.8324 | AUPRC: 0.4729



In [12]:
# Now evaluate the model
avg_loss, aucroc, auprc = evaluate_model(model_max_pooling, test_loader, criterion, device)
print(f"Test Loss: {avg_loss:.4f}, AUC-ROC: {aucroc:.4f}, AUC-PRC: {auprc:.4f}")

Evaluation - Loss: 0.3227 - AUCROC: 0.8310 - AUPRC: 0.4833
Test Loss: 0.3227, AUC-ROC: 0.8310, AUC-PRC: 0.4833


# LSTM w/ Average

In [13]:
class LSTM_Model_Pooling(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=1, dropout=0.3):
        super(LSTM_Model_Pooling, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,       # 40 features per time step
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)           # out: (batch_size, seq_len, hidden_size)
        out = out.mean(dim=1)           # Pooling: (batch_size, hidden_size)   
        out = self.fc(out)              # (batch_size, num_classes)
        return out.squeeze()            # (batch_size,) for BCEWithLogitsLoss

In [14]:
# Use the previous data loaders and train the new model
model_pooling = LSTM_Model_Pooling(input_size=input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_pooling.parameters(), lr=0.001)

model_pooling = train_model_with_validation(model_pooling, train_loader, validation_loader, criterion, optimizer, device)

Epoch 1/20
  Train Loss: 0.4447 | AUCROC: 0.5207 | AUPRC: 0.1536
  Val   Loss: 0.4115 | AUCROC: 0.6489 | AUPRC: 0.2070



Epoch 2/20
  Train Loss: 0.4031 | AUCROC: 0.5151 | AUPRC: 0.1502
  Val   Loss: 0.4050 | AUCROC: 0.7337 | AUPRC: 0.2962



Epoch 3/20
  Train Loss: 0.3954 | AUCROC: 0.6053 | AUPRC: 0.2157
  Val   Loss: 0.3823 | AUCROC: 0.7647 | AUPRC: 0.3821



Epoch 4/20
  Train Loss: 0.3787 | AUCROC: 0.6537 | AUPRC: 0.2958
  Val   Loss: 0.3634 | AUCROC: 0.7765 | AUPRC: 0.4010



Epoch 5/20
  Train Loss: 0.3541 | AUCROC: 0.7423 | AUPRC: 0.3604
  Val   Loss: 0.3518 | AUCROC: 0.8021 | AUPRC: 0.4287



...

Epoch 16/20
  Train Loss: 0.3175 | AUCROC: 0.8174 | AUPRC: 0.4530
  Val   Loss: 0.3225 | AUCROC: 0.8347 | AUPRC: 0.4888



Epoch 17/20
  Train Loss: 0.3133 | AUCROC: 0.8243 | AUPRC: 0.4659
  Val   Loss: 0.3117 | AUCROC: 0.8382 | AUPRC: 0.4926



Epoch 18/20
  Train Loss: 0.3130 | AUCROC: 0.8236 | AUPRC: 0.4627
  Val   Loss: 0.3352 | AUCROC: 0.8378 | AUPRC: 0.4928



Epoch 19/20
  Train Loss: 0.3165 | AUCROC: 0.8167 | AUPRC: 0.4584
  Val   Loss: 0.3131 | AUCROC: 0.8343 | AUPRC: 0.4834



Epoch 20/20
  Train Loss: 0.3119 | AUCROC: 0.8264 | AUPRC: 0.4595
  Val   Loss: 0.3085 | AUCROC: 0.8405 | AUPRC: 0.4977



In [15]:
# Now evaluate the model
avg_loss, aucroc, auprc = evaluate_model(model_pooling, test_loader, criterion, device)
print(f"Test Loss: {avg_loss:.4f}, AUC-ROC: {aucroc:.4f}, AUC-PRC: {auprc:.4f}")

Evaluation - Loss: 0.3115 - AUCROC: 0.8419 - AUPRC: 0.5058
Test Loss: 0.3115, AUC-ROC: 0.8419, AUC-PRC: 0.5058


# Bidirectional LSTM

In [16]:
class LSTM_Model_Bi(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=1, dropout=0.3):
        super(LSTM_Model_Bi, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,       # 41 features per time step
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes) # *2 for bidirectional

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)           # out: (batch_size, seq_len, hidden_size)
        out = out[:, -1, :]             # Take last time step: (batch_size, hidden_size)
        out = self.fc(out)              # (batch_size, num_classes)
        return out.squeeze()            # (batch_size,) for BCEWithLogitsLoss

In [17]:
# Train the model
model_bi = LSTM_Model_Bi(input_size=input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_bi.parameters(), lr=0.001)

model_bi = train_model_with_validation(model_bi, train_loader, validation_loader, criterion, optimizer, device)

Epoch 1/20
  Train Loss: 0.4464 | AUCROC: 0.4885 | AUPRC: 0.1379
  Val   Loss: 0.4131 | AUCROC: 0.5269 | AUPRC: 0.1432



Epoch 2/20
  Train Loss: 0.4029 | AUCROC: 0.5108 | AUPRC: 0.1472
  Val   Loss: 0.4062 | AUCROC: 0.7206 | AUPRC: 0.3183



Epoch 3/20
  Train Loss: 0.3754 | AUCROC: 0.6939 | AUPRC: 0.2851
  Val   Loss: 0.3631 | AUCROC: 0.8014 | AUPRC: 0.4216



Epoch 4/20
  Train Loss: 0.3336 | AUCROC: 0.7833 | AUPRC: 0.4281
  Val   Loss: 0.3216 | AUCROC: 0.8211 | AUPRC: 0.4566



Epoch 5/20
  Train Loss: 0.3325 | AUCROC: 0.7896 | AUPRC: 0.4184
  Val   Loss: 0.3242 | AUCROC: 0.8231 | AUPRC: 0.4678



...

Epoch 16/20
  Train Loss: 0.3051 | AUCROC: 0.8334 | AUPRC: 0.4889
  Val   Loss: 0.3062 | AUCROC: 0.8439 | AUPRC: 0.5130



Epoch 17/20
  Train Loss: 0.3065 | AUCROC: 0.8315 | AUPRC: 0.4893
  Val   Loss: 0.3076 | AUCROC: 0.8442 | AUPRC: 0.5131



Epoch 18/20
  Train Loss: 0.3063 | AUCROC: 0.8306 | AUPRC: 0.4926
  Val   Loss: 0.3069 | AUCROC: 0.8410 | AUPRC: 0.5138



Epoch 19/20
  Train Loss: 0.3059 | AUCROC: 0.8318 | AUPRC: 0.4912
  Val   Loss: 0.3114 | AUCROC: 0.8417 | AUPRC: 0.5137



Epoch 20/20
  Train Loss: 0.3043 | AUCROC: 0.8328 | AUPRC: 0.4998
  Val   Loss: 0.3092 | AUCROC: 0.8389 | AUPRC: 0.5066



In [18]:
# Now evaluate
avg_loss, aucroc, auprc = evaluate_model(model_bi, test_loader, criterion, device)
print(f"Test Loss: {avg_loss:.4f}, AUC-ROC: {aucroc:.4f}, AUC-PRC: {auprc:.4f}")

Evaluation - Loss: 0.3580 - AUCROC: 0.8220 - AUPRC: 0.4549
Test Loss: 0.3580, AUC-ROC: 0.8220, AUC-PRC: 0.4549


# Transformer

In [19]:
import torch.nn

class TransformerClassifier(nn.Module):
    def __init__(self, input_size, num_classes=1, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.3):
        super().__init__()
        self.input_size = input_size

        # Project input features to model dimension
        self.embedding = nn.Linear(input_size, dim_feedforward)

        # Positional Encoding
        self.pos_encoder = PositionalEncoding(dim_feedforward, dropout)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim_feedforward,
            nhead=nhead,
            dim_feedforward=dim_feedforward * 2,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Final classifier
        self.fc = nn.Linear(dim_feedforward, num_classes)

    def forward(self, x):
        # x: (batch, seq_len, input_size)

        if x.dim() == 2:
            x = x.unsqueeze(1) # (batch, 1, input_size)

        x = self.embedding(x)                # (batch, seq_len, d_model)
        #print("After embedding:", x.shape)  # Debug print
        x = self.pos_encoder(x)
        #print("After pos encoding:", x.shape)  # Debug print
        x = self.transformer_encoder(x)      # (batch, seq_len, d_model)
        #print("After transformer encoder:", x.shape)

        x = x.mean(dim=1)                    # mean pooling over time
        #print("After pooling:", x.shape)     # Debug print
        out = self.fc(x).squeeze()           # (batch,)
        #print("After fc:", out.shape)        # Debug print
        return out


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=500):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(torch.log(torch.tensor(10000.0)) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [20]:
# Train the Transformer model
model_transformer = TransformerClassifier(input_size=input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_transformer.parameters(), lr=0.001)

model_transformer = train_model_with_validation(model_transformer, train_loader, validation_loader, criterion, optimizer, device)

Epoch 1/20
  Train Loss: 0.4176 | AUCROC: 0.5097 | AUPRC: 0.1479
  Val   Loss: 0.4065 | AUCROC: 0.7721 | AUPRC: 0.3961



Epoch 2/20
  Train Loss: 0.3876 | AUCROC: 0.6355 | AUPRC: 0.2363
  Val   Loss: 0.3494 | AUCROC: 0.8069 | AUPRC: 0.4529



Epoch 3/20
  Train Loss: 0.3473 | AUCROC: 0.7585 | AUPRC: 0.3659
  Val   Loss: 0.3390 | AUCROC: 0.8218 | AUPRC: 0.4722



Epoch 4/20
  Train Loss: 0.3355 | AUCROC: 0.7794 | AUPRC: 0.4146
  Val   Loss: 0.3205 | AUCROC: 0.8281 | AUPRC: 0.4816



Epoch 5/20
  Train Loss: 0.3304 | AUCROC: 0.7913 | AUPRC: 0.4205
  Val   Loss: 0.3369 | AUCROC: 0.8301 | AUPRC: 0.4877



...

Epoch 16/20
  Train Loss: 0.3109 | AUCROC: 0.8260 | AUPRC: 0.4723
  Val   Loss: 0.3064 | AUCROC: 0.8428 | AUPRC: 0.5159



Epoch 17/20
  Train Loss: 0.3113 | AUCROC: 0.8211 | AUPRC: 0.4848
  Val   Loss: 0.3408 | AUCROC: 0.8427 | AUPRC: 0.5155



Epoch 18/20
  Train Loss: 0.3112 | AUCROC: 0.8258 | AUPRC: 0.4720
  Val   Loss: 0.3095 | AUCROC: 0.8440 | AUPRC: 0.5132



Epoch 19/20
  Train Loss: 0.3122 | AUCROC: 0.8227 | AUPRC: 0.4742
  Val   Loss: 0.3218 | AUCROC: 0.8440 | AUPRC: 0.5144



Epoch 20/20
  Train Loss: 0.3084 | AUCROC: 0.8304 | AUPRC: 0.4771
  Val   Loss: 0.3070 | AUCROC: 0.8415 | AUPRC: 0.5178



In [21]:
# Evaluate the model
avg_loss, aucroc, auprc = evaluate_model(model_transformer, test_loader, criterion, device)
print(f"Test Loss: {avg_loss:.4f}, AUC-ROC: {aucroc:.4f}, AUC-PRC: {auprc:.4f}")

Evaluation - Loss: 0.3196 - AUCROC: 0.8426 - AUPRC: 0.5114
Test Loss: 0.3196, AUC-ROC: 0.8426, AUC-PRC: 0.5114


# Q2.3 Tokenizing 

In [ ]:
# For this part we need the scaled data with simple
set_a_initial, set_b_initial, set_c_initial = set_a_scaled, set_b_scaled, set_c_scaled
set_a_scaled.shape, set_b_scaled.shape, set_c_scaled.shape

((183416, 42), (183495, 42), (183711, 42))

## Create the TZV Dataframe (following Horn et al.)

In [ ]:
import pandas as pd
def build_TZV_dataframe(original_df, label_df, base_time="2025-03-10 00:00:00", duration_hours=48): 
    """ 
        Build a long-format dataframe with columns [T, Z, V, y] from an original wide dataframe.
        Parameters:
            original_df (pd.DataFrame): DataFrame with columns [RecordID, Time, f1, f2, ..., f41] (already scaled).
            label_df (pd.DataFrame): DataFrame with columns [RecordID, y] containing the label for each RecordID.
            base_time (str): Base time used for normalizing the Time column.
            duration_hours (int): The duration (in hours) from base_time over which Time is normalized (here, 48 hours).

        Returns:
            long_df (pd.DataFrame): Long-format dataframe with columns:
                                    T: normalized time [0, 1],
                                    Z: index of the feature,
                                    V: measurement value,
                                    y: label corresponding to RecordID.
            feature_to_index (dict): Mapping from original feature names to integer indices.
    """
    # Merge the labels with the original dataframe using RecordID.
    df = original_df.copy().merge(label_df, on="RecordID", how="left")

    # Convert Time to datetime and compute normalized time T.
    df["Time"] = pd.to_datetime(df["Time"])
    start_time = pd.to_datetime(base_time)
    end_time = start_time + pd.Timedelta(hours=duration_hours)
    total_seconds = (end_time - start_time).total_seconds()
    df["T"] = (df["Time"] - start_time).dt.total_seconds() / total_seconds

    # Identify feature columns: all columns except RecordID, Time, T, and the label column.
    # Here, assuming the label column is named "In-hospital_death".
    feature_cols = [col for col in df.columns if col not in ["RecordID", "Time", "T", "In-hospital_death"]]

    # Since the features are already scaled, we skip the scaling step.

    # Melt the dataframe from wide to long format.
    long_df = pd.melt(df, id_vars=["T", "In-hospital_death"], value_vars=feature_cols, 
                    var_name="Z", value_name="V")

    # Map feature names to indices for the "Z" column.
    feature_to_index = {feat: idx for idx, feat in enumerate(feature_cols)}
    long_df["Z"] = long_df["Z"].map(feature_to_index)

    # Sort the final dataframe by normalized time T and reset the index.
    long_df = long_df.sort_values("T").reset_index(drop=True)
    long_df = long_df.dropna(subset=["V"])

    return long_df, feature_to_index

In [ ]:
# Build the TZV dataframes
TZV_a, feature_to_index_a = build_TZV_dataframe(set_a_initial, death_a)
TZV_b, feature_to_index_b = build_TZV_dataframe(set_b_initial, death_b)
TZV_c, feature_to_index_c = build_TZV_dataframe(set_c_initial, death_c)

print(TZV_a.shape)
TZV_a.head(10)

In [ ]:
# Check for the total number of not NaN values under some specified columns
selected_cols = [col for col in set_a_initial.columns if col not in ["RecordID", "Time"]]
set_a_initial[selected_cols].notna().sum().sum()

Checked that the number of not NaN values is the same as the rows of the new dataframe! Let's go
(We have to believe in this format)

## Train the TZV Format with a Transformer

In [ ]:
# Remove the In-hospital_death column from the TZV dataframes, but save it
y_a = TZV_a.pop("In-hospital_death")
y_b = TZV_b.pop("In-hospital_death")
y_c = TZV_c.pop("In-hospital_death")

# Convert the TZV dataframes to PyTorch tensors
X_a = torch.tensor(TZV_a[["T", "Z", "V"]].values, dtype=torch.float32)
X_b = torch.tensor(TZV_b[["T", "Z", "V"]].values, dtype=torch.float32)
X_c = torch.tensor(TZV_c[["T", "Z", "V"]].values, dtype=torch.float32)
print(X_a.shape, X_b.shape, X_c.shape)

# Create the datasets and dataloaders
from torch.utils.data import TensorDataset

dataset_a = TensorDataset(X_a, torch.tensor(y_a.values, dtype=torch.float32))
dataset_b = TensorDataset(X_b, torch.tensor(y_b.values, dtype=torch.float32))
dataset_c = TensorDataset(X_c, torch.tensor(y_c.values, dtype=torch.float32))

loader_a = DataLoader(dataset_a, batch_size=64, shuffle=True)
loader_b = DataLoader(dataset_b, batch_size=64, shuffle=False)
loader_c = DataLoader(dataset_c, batch_size=64, shuffle=False)

In [ ]:
model_tvz = TransformerClassifier(input_size=3).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_tvz.parameters(), lr=0.001)

model_tvz = train_model_with_validation(model_tvz, loader_a, loader_b, criterion, optimizer, device)